In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 13.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import openai

openai.api_key = "API_KEY"

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/toxic_dataset_en.csv")

In [ ]:
import time
def translate_chat(text):
   try:

      response = openai.ChatCompletion.create(
      model = "gpt-3.5-turbo",
      messages = [{"role": "user", "content":f'Translate \"{text}\" into Spanish with Chilean slang from the perspective of a school student. Please don\'t add any explanations for the translation.'}],
      temperature=0.5  # Adjust the temperature value as desired

      )
      answer = response["choices"][0]["message"]["content"]
      return answer


   except openai.error.RateLimitError as e:

      time.sleep(60)

      return translate_chat(text)

   except openai.error.APIError as e:
      time.sleep(60)

      return translate_chat(text)


   except openai.error.ServiceUnavailableError as e:

          time.sleep(60)

          return translate_chat(text)

In [ ]:
def flow_from_df(dataframe: pd.DataFrame, chunk_size: int = 10, start_index=0):
    for start_row in range(start_index, dataframe.shape[0], chunk_size):
        end_row  = min(start_row + chunk_size, dataframe.shape[0])
        yield dataframe.iloc[start_row:end_row, :]

In [ ]:
new_df = pd.read_csv("/content/drive/MyDrive/toxic_dataset_cl.csv")

from tqdm.notebook import tqdm
import asyncio
import concurrent.futures
count = 1

BATCH_SIZE = 5

for rows in tqdm(flow_from_df(df, BATCH_SIZE, len(new_df)), total=((len(df) - len(new_df)+BATCH_SIZE - 1) // BATCH_SIZE)):
  loop = asyncio.get_event_loop()

  futures = []

  with concurrent.futures.ThreadPoolExecutor() as pool:
    for i, row in rows.iterrows():
      en_text = row.comment_text
      toxic = row.true_toxic
      futures.append(loop.run_in_executor(pool, lambda: (en_text, translate_chat(en_text), toxic)))
    results = await asyncio.gather(*futures)
  for en_text, cl_text, toxic in results:
    new_df = pd.concat([new_df, pd.DataFrame([[en_text, cl_text, toxic]], columns=["comment_text", "chilean_text", "true_toxic"])]).reset_index(drop=True)
  new_df.to_csv("/content/drive/MyDrive/toxic_dataset_cl.csv")

new_df

  0%|          | 0/3123 [00:00<?, ?it/s]

,Unnamed: 0.8,Unnamed: 0.7,Unnamed: 0.6,Unnamed: 0.5,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,comment_text,chilean_text,true_toxic
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,"Jerskine, if you clicked on Sally Regenhard yo...","Jerskine, si pinchaste en Sally Regenhard, cac...",0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,"If you're a motherf**king a**hole, what can I ...","""Si eres un conch* de tu madre, ¿qué puedo hac...",1
2,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,Once\nThe song is due to chart at #1 in the UK...,Una vez que la canción va a ser número uno en ...,1
3,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,you reported me for nothing and that kew garde...,"""Me denunciaste por nada y ese loco de kew gar...",1
4,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,"rm speedy, not quite sure it's that badly spam...","""rm rápido, no toy seguro si es tan spam. jaja...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
32445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Delete the page and shut up,Borra la página y cállate.,0
32446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""\n Thanks. Indeed it was the incorrect name r...","Vale. De hecho, estaba mal el nombre desde el ...",0
32447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Raw Materials of a Cell Phone \nI was wonderin...,"""Loco, ¿alguien puede tirar los materiales bru...",0
32448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""\nOkay, I will think about it. Perhaps """"Othe...","Dale, lo voy a pensar. Quizás ""Otras Ritos Ori...",0
